# Envy-free allocation checker

Envy-free indivisible item allocation is a fair division problem. If none of the agents believe any other agent got more value allocated than themselves, it is envy-free. Algorithmicaly checking a specific allocation is trivial. However, checking whether an envy-free allocation is possible is considerably more complex in certain cases. This problem is also explored but in a heuristic way. The functionality is rather simplistic as I only needed this much while working on a course assignment.

# Functions

In [1]:
import pandas as pd
import numpy as np
import random


def create_random_valuations(
    n_agents,
    n_items,
    max_value,
    min_value=0,
    allow_item_equality=True,
    agent_names=None,
    item_names=None,
):
    """Creates random sets of valuations for each agent based on the desired numbers of 
    agents and items.

    Args:
        n_agents (int): Number of agents in the allocation.
        n_items (int): Number of items in the allocation.
        max_value (int): Maximum desired value of an item for an agent.
        min_value (int, optional): Minimum desired value of an item for an agent. 
            Defaults to 0.
        allow_item_equality (bool, optional): Indicates whether an agent can have the 
            same valuation for multiple items. Defaults to True.
        agent_names (list, optional): List of agent names that will overwrite the 
            DataFrame row names. Defaults to None.
        item_names (list, optional): List of item names that will overwrite the 
            DataFrame column names. Defaults to None.

    Returns:
        DataFrame: A valuations DataFrame where rows represent the agents and columns 
            represent the items.
    """
    min_value = int(min_value)
    max_value = int(max_value)

    if agent_names is not None and n_agents != len(agent_names):
        agent_names = None
    if item_names is not None and n_items != len(item_names):
        item_names = None

    if allow_item_equality and n_items + 1 >= (max_value - min_value):
        valuations = pd.DataFrame(
            np.random.randint(min_value, max_value, size=(n_agents, n_items))
        )
    else:
        valuations = pd.DataFrame(index=range(n_agents), columns=range(n_items))
        for row_id, row in valuations.iterrows():
            valuations.iloc[row_id, :] = random.sample(
                list(range(min_value, max_value + 1)), n_items
            )

    if agent_names:
        valuations.index = agent_names

    if item_names:
        valuations.columns = item_names

    return valuations


def create_random_allocations(
    valuations, min_items=1, max_items=None, force_allocate_all=False
):
    """Randomly allocates the items to agents based on the agents and items in the 
    valuations DataFrame.

    Args:
        valuations (DataFrame): Valuations DataFrame where rows represent the agents and
            columns represent the items.
        min_items (int, optional): Minimum number of items to be allocated for each 
            agent. Defaults to 1.
        max_items (int|None, optional): Maximum number of items to be allocated for each 
            agent. Defaults to None (without a limit).
        force_allocate_all (bool, optional): Indicates whether all items must be 
            allocated to an agent. Defaults to False.

    Returns:
        dict: Allocations dictionary where keys are the agents and values are the 
            lists of items allocated for each agent.
    """
    n_agents, n_items = valuations.shape
    min_items = int(max(0, min_items))
    if max_items is None:
        max_items = n_items

    done = False

    while not done:
        allocations = {}
        if min_items > 0 and n_items < n_agents:
            return allocations

        available = valuations.columns.tolist()
        for agent_id in random.sample(list(range(n_agents)), n_agents):
            n_available = len(available)
            n_available_allocation = min(n_available, max_items)
            if n_available_allocation < min_items:
                n_available_allocation = min_items
            n_allocated = random.randint(min_items, n_available_allocation)
            if n_available < n_allocated:
                n_allocated = n_available
            allocation = sorted(random.sample(available, n_allocated))
            available = [item for item in available if item not in allocation]
            allocations[valuations.index[agent_id]] = allocation

        if (not force_allocate_all) or (
            force_allocate_all
            and sum([len(v) for v in allocations.values()]) >= n_items
        ):
            done = True

    return dict(sorted(allocations.items(), key=lambda item: item[0]))


def is_envy_free(valuations, allocations):
    """Checks whether an allocation is envy-free based on the provided valuations.

    Args:
        valuations (DataFrame): Valuations DataFrame where rows represent the agents and
            columns represent the items.
        allocations (dict): Allocations dictionary where keys are the agents and values 
            are the lists of items allocated for each agent.

    Returns:
        bool: Boolean indicating whether the allocation is envy-free.
    """
    for row_id, (row_name, row) in enumerate(valuations.iterrows()):
        subjective_allocation_utilities = {agent: 0 for agent in valuations.index}

        for row_id_2, (row_name_2, row_2) in enumerate(valuations.iterrows()):
            agent = valuations.index[row_id_2]
            utility = sum([row[item] for item in allocations[agent]])
            subjective_allocation_utilities[agent] = utility

        if subjective_allocation_utilities[valuations.index[row_id]] == max(
            subjective_allocation_utilities.values()
        ):
            pass
        else:
            return False

    return True


def is_envy_freeness_possible(
    valuations, n_simulations, min_items=1, max_items=None, force_allocate_all=False
):
    """Heuristically checks whether it is possible to have an envy-free allocation for 
    the given valuations. It generates random allocations until an envy-free one is 
    found or the simulation limit is reached. Therefore, the conditions are stricter 
    (allocating all items, not allowing valuations of 0, etc.), it is more likely to 
    give a false negative as the valuation complexity grows.

    Args:
        valuations (int): Valuations DataFrame where rows represent the agents and
            columns represent the items.
        n_simulations (int): Number of times a random set of allocations will be 
            generated and checked before it returns False.
        min_items (int, optional): Minimum number of items to be allocated for each 
            agent. Defaults to 1.
        max_items (int|None, optional): Maximum number of items to be allocated for each 
            agent. Defaults to None (without a limit).
        force_allocate_all (bool, optional): Indicates whether all items must be 
            allocated to an agent. Defaults to False.

    Returns:
        (bool, dict): Tuple of a boolean indicating whether the allocation is envy-free 
            and a dictionary that shows an example envy-free allocation, along with the 
            matching simulation's index.
    """
    for i in range(n_simulations):
        random_allocations = create_random_allocations(
            valuations,
            min_items=min_items,
            max_items=max_items,
            force_allocate_all=force_allocate_all,
        )
        test = is_envy_free(valuations, random_allocations)
        if test:
            return (
                True,
                {"example_allocation": random_allocations, "match_sim_index": i},
            )

    return (False, {})

# Use

Creating some random valuations:

In [2]:
valuations = create_random_valuations(n_agents=3, n_items=3, max_value=4, min_value=0, allow_item_equality=True, agent_names=["Agent 1", "Agent 2", "Agent 3"], item_names=["A", "B", "C"])

valuations

,A,B,C
Agent 1,1,3,0
Agent 2,2,2,3
Agent 3,2,2,2


Randomly allocating the previously appraised items:

In [3]:
allocations = create_random_allocations(valuations, min_items=1, max_items=2, force_allocate_all=True)

allocations

{'Agent 1': [], 'Agent 2': ['A', 'B'], 'Agent 3': ['C']}

We can now check whether this allocation is envy-free:

In [4]:
is_envy_free(valuations, allocations)

False

Heuristically checking if it is possible (and if so, how) to have an envy-free allocation with the given valuations:

In [5]:
is_envy_freeness_possible(valuations, n_simulations=1000, min_items=1, max_items=2, force_allocate_all=True)

(True,
 {'example_allocation': {'Agent 1': ['B'], 'Agent 2': ['C'], 'Agent 3': ['A']},
  'match_sim_index': 9})